In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

KeyboardInterrupt: 

In [ ]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_일반')
BASE_YYMM = '201912'
BASE_YYYY = BASE_YYMM[:4]

In [ ]:
# 데이터 불러오기
일반_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'OGL_ELP_PRM': float})
일반_출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RN_ELP_PRM': float})
일반_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'OGL_LOSS': float})
일반_출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'RN_LOSS': float})
일반_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'ELP_PRM': float, 'RN_ELP_PRM': float}) \
    .rename(columns={'ELP_PRM': 'OGL_ELP_PRM'}) # TODO: ELP_PRM -> OGL_ELP_PRM로 수정하기
일반_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
일반_특약수수료 = pd.read_excel(FILE_PATH / f'일반_특약수수료_{202012}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_특약정보 = pd.read_excel(FILE_PATH / f'일반_특약정보_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
산업_손해율_변동계수 = pd.read_excel(FILE_PATH / '산업_손해율_변동계수.xlsx')

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format

# 특약정보
## 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
일반_특약정보_가공 = 일반_특약정보.copy()
일반_특약정보_가공['TTY_CD_GRP'] = None
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("(재물보험|패키지)"), 'TTY_CD_GRP'] = '재물보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("기술보험"), 'TTY_CD_GRP'] = '기술보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("근재보험"), 'TTY_CD_GRP'] = '근재보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("배상책임보험"), 'TTY_CD_GRP'] = '배상책임보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("해외PST"), 'TTY_CD_GRP'] = '해외PST'

# 특약수수료
일반_특약수수료_가공 = 일반_특약수수료.copy()
일반_특약수수료_가공 = 일반_특약수수료_가공.merge(일반_특약정보_가공, on=['RRNR_TTY_CD', 'TTY_YR'], how='left').query('~TTY_CD_GRP.isna()')
일반_특약수수료_가공 = 일반_특약수수료_가공[['TTY_CD_GRP', 'TTY_YR', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'PVSN_CMSN_RT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']].drop_duplicates()

# 직전3년평균손해율
일반_원수_직전3년연간경과보험료_가공 = 일반_원수_직전3년연간경과보험료.copy()
일반_원수_직전3년연간경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간경과보험료_가공, 일반_상품정보)
일반_원수_직전3년연간경과보험료_집계 = 일반_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

일반_출재_직전3년연간경과보험료_가공 = 일반_출재_직전3년연간경과보험료.copy()
일반_출재_직전3년연간경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간경과보험료_가공)
일반_출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간경과보험료_가공, 일반_상품정보)
일반_출재_직전3년연간경과보험료_집계 = 일반_출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_ELP_PRM']].sum()

일반_원수_직전3년연간손해액_가공 = 일반_원수_직전3년연간손해액.copy()
일반_원수_직전3년연간손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간손해액_가공, 일반_상품정보)
일반_원수_직전3년연간손해액_집계 = 일반_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

일반_출재_직전3년연간손해액_가공 = 일반_출재_직전3년연간손해액.copy()
일반_출재_직전3년연간손해액_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간손해액_가공)
일반_출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간손해액_가공, 일반_상품정보)
일반_출재_직전3년연간손해액_집계 = 일반_출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_LOSS']].sum()

일반_직전3년평균손해율 = 일반_원수_직전3년연간경과보험료_집계 \
    .merge(일반_출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)
일반_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - RN_ELP_PRM', inplace=True)
일반_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - RN_LOSS', inplace=True)
일반_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
일반_직전3년평균손해율 = 일반_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()

# 특약보종별 직전1년경과보험료
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료.copy()
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년경과보험료_가공, 일반_상품정보)
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()

# 특약보종별 직전1년손해액
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액.copy()
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년손해액_가공, 일반_상품정보)
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_LOSS', 'RN_LOSS']].sum()

일반_특약보종별_실적 = 일반_특약보종별_직전1년경과보험료_가공 \
    .merge(일반_특약보종별_직전1년손해액_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')
일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)

# 특약보종별 실적 집계
# 일반_특약보종별_실적_가공 = []
# for tty_cd_grp in ['재물보험특약', '기술보험특약', '근재보험특약', '배상책임보험특약', '해외PST']:
#     STRT_YYYY = str(int(BASE_YYYY)  - (7 if tty_cd_grp in ['근재보험특약', '배상책임보험특약'] else 5))
#     일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR > @STRT_YYYY'))
# 일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_실적_가공 = []
for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
    STRT_YYYY = str(int(BASE_YYYY)  - (7 if boz_cd in ['A005', 'A006'] else 5))
    일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('BOZ_CD == @boz_cd').query('TTY_YR > @STRT_YYYY'))
일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)

C:\Users\11700205.EDBINS\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
pd.options.display.float_format = '{:,.0f}'.format

# 보유리스크율
## 위험계수적용법
일반_보유리스크율_위험계수적용법 = []
for tty_cd_grp in ['근재보험특약', '기술보험특약', '재물보험특약', '배상책임보험특약', '해외PST']:
    for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
        prem = 일반_특약보종별_실적_가공.query('TTY_CD_GRP == @tty_cd_grp').query('BOZ_CD == @boz_cd').groupby('TTY_YR', as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()
        if len(prem) == 0: continue
        comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').drop('TTY_CD_GRP', axis=1)
        loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        risk_coef = 0.55 # TODO: boz_cd별 적용위험계수로 수정
        일반_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef(comm, prem, loss_ratio, risk_coef)
        일반_보유리스크율_위험계수적용법_특약보종별.insert(0, 'BOZ_CD', boz_cd)
        일반_보유리스크율_위험계수적용법_특약보종별.insert(0, 'TTY_CD_GRP', tty_cd_grp)
        일반_보유리스크율_위험계수적용법.append(일반_보유리스크율_위험계수적용법_특약보종별)
일반_보유리스크율_위험계수적용법 = pd.concat(일반_보유리스크율_위험계수적용법, axis=0)
일반_보유리스크율_위험계수적용법.query('OGL_ELP_PRM > 0', inplace=True)

In [ ]:
## 손해율분포법
# input
ogl_elp_prm_tty = 20000000
rn_elp_prm_tty = 2000000
ogl_loss_tty = 8000000
rn_loss_tty = 1200000
ogl_1yr_elp_prm_tty_boz = 1000000
rn_1yr_elp_prm_tty_boz = 250000
ogl_1yr_loss_tty_boz = 1200000
rn_1yr_loss_tty_boz = 200000
loss_ratio = .5
cv = 0.118124992066769
slope, a, b, top, bottom = 0.8, 0.6, 0.3, 0.6, 0.3

get_ret_risk_rate_by_loss_dist(ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty, 
    ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz, 
    loss_ratio, cv, slope, a, b, top, bottom)

(23381.602069041834, 22212.52196558949)